In [56]:
from model import rotary
import torch
from einops import rearrange
import torch.nn.functional as F
torch.manual_seed(0)
#rotary.Rotary(config.model.hidden_size // config.model.n_heads)
hidden_size=4
n_heads=2
seq_len=3
batch_size=1
rotary_emb=rotary.Rotary(hidden_size // n_heads)
x=torch.rand([batch_size,seq_len,hidden_size]) #batch of size 1, sequence length 3, embedded into the hidden size 4. Embedding is all .1
rotary_cos_sin = rotary_emb(x)


In [57]:
print(x)
print(rotary_cos_sin[0])
print(rotary_cos_sin[1])

tensor([[[0.4963, 0.7682, 0.0885, 0.1320],
         [0.3074, 0.6341, 0.4901, 0.8964],
         [0.4556, 0.6323, 0.3489, 0.4017]]])
tensor([[[[[ 1.0000,  1.0000]],

          [[ 1.0000,  1.0000]],

          [[ 1.0000,  1.0000]]],


         [[[ 0.5403,  0.5403]],

          [[ 0.5403,  0.5403]],

          [[ 1.0000,  1.0000]]],


         [[[-0.4161, -0.4161]],

          [[-0.4161, -0.4161]],

          [[ 1.0000,  1.0000]]]]])
tensor([[[[[0.0000, 0.0000]],

          [[0.0000, 0.0000]],

          [[0.0000, 0.0000]]],


         [[[0.8415, 0.8415]],

          [[0.8415, 0.8415]],

          [[0.0000, 0.0000]]],


         [[[0.9093, 0.9093]],

          [[0.9093, 0.9093]],

          [[0.0000, 0.0000]]]]])


In [58]:
qkv = torch.rand([batch_size,seq_len,3*hidden_size]) 
qkv = rearrange(qkv, 'b s (three h d) -> b s three h d', three=3, h=2)

In [59]:
for i in range(3):
    print(qkv[:, :, i])

tensor([[[[0.0223, 0.1689],
          [0.2939, 0.5185]],

         [[0.4194, 0.5529],
          [0.9527, 0.0362]],

         [[0.2081, 0.9298],
          [0.7231, 0.7423]]]])
tensor([[[[0.6977, 0.8000],
          [0.1610, 0.2823]],

         [[0.1852, 0.3734],
          [0.3051, 0.9320]],

         [[0.5263, 0.2437],
          [0.5846, 0.0332]]]])
tensor([[[[0.6816, 0.9152],
          [0.3971, 0.8742]],

         [[0.1759, 0.2698],
          [0.1507, 0.0317]],

         [[0.1387, 0.2422],
          [0.8155, 0.7932]]]])


In [60]:
with torch.cuda.amp.autocast(enabled=False):
            cos, sin = rotary_cos_sin
            qkv = rotary.apply_rotary_pos_emb(
                qkv, cos.to(qkv.dtype), sin.to(qkv.dtype)
            )

C:\Users\bezem\AppData\Local\Temp\ipykernel_8316\1714280401.py:1: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


In [61]:
q, k, v = qkv[:, :, 0], qkv[:, :, 1], qkv[:, :, 2]

In [62]:
q

tensor([[[[ 0.0223,  0.1689],
          [ 0.2939,  0.5185]],

         [[-0.2386,  0.6517],
          [ 0.4843,  0.8212]],

         [[-0.9321, -0.1977],
          [-0.9759,  0.3486]]]])

In [63]:
k

tensor([[[[ 0.6977,  0.8000],
          [ 0.1610,  0.2823]],

         [[-0.2141,  0.3576],
          [-0.6194,  0.7603]],

         [[-0.4406,  0.3772],
          [-0.2734,  0.5178]]]])

In [64]:
v

tensor([[[[0.6816, 0.9152],
          [0.3971, 0.8742]],

         [[0.1759, 0.2698],
          [0.1507, 0.0317]],

         [[0.1387, 0.2422],
          [0.8155, 0.7932]]]])

In [65]:
x = F.scaled_dot_product_attention(
                q, k, v,attn_mask=None
            )

In [66]:
x = rearrange(x, 'b s h d -> b s (h d)', b=batch_size)

In [67]:
x

tensor([[[0.5444, 0.8954, 0.5606, 0.8977],
         [0.1617, 0.1357, 0.1627, 0.1451],
         [0.4552, 0.4998, 0.4634, 0.5066]]])

We need to try doing the same thing, but instead of <br>

qkv = rotary.apply_rotary_pos_emb(
                qkv, cos.to(qkv.dtype), sin.to(qkv.dtype)
            )<br>
do<br>
<br>
import flash_attn.layers.rotary<br>
cos.to(qkv.dtype)<br>
sin.to(qkv.dtype)<br>
cos = cos[0,:,0,0,:cos.shape[-1]//2]<br>
sin = sin[0,:,0,0,:sin.shape[-1]//2]<br>
flash_attn.layers.rotary.apply_rotary_emb_qkv_(qkv, cos, sin)<br>
<br>
If after this qkv is different, then the issue is with their rotary embedding implementation.<br>
<br>
We also need to instead of <br>
<br>
x = F.scaled_dot_product_attention(<br>
                q, k, v,attn_mask=None<br>
            )<br>
do<br>
<br>
qkv = rearrange(qkv, 'b s ... -> (b s) ...')<br>
cu_seqlens = torch.arange(<br>
                0, (batch_size + 1) * seq_len, step=seq_len,<br>
                dtype=torch.int32, device=qkv.device)<br>
from flash_attn.flash_attn_interface import flash_attn_varlen_qkvpacked_func<br>
x = flash_attn_varlen_qkvpacked_func(<br>
            qkv, cu_seqlens, seq_len, 0., causal=False)<br>
<br>
if x is different then the issue is my adaptation of flash attn to standard attn<br>
